# 1.1 урок

## Семинар Основы статистики

### Задача 1. Оценка вероятностей


40% заказов имеют стоимость больше 1000 рублей каждый. Допустим, новый заказ стоит больше 1000 рублей с вероятностью 0.4. Оцените вероятность того, что из 100 новых заказов 50 или более стоят дороже 1000 рублей.

Ответ округлите с точность до 3 знака после точки.

In [3]:
import numpy as np

# Cгенерируем достаточно большое количество исходов
values = np.random.binomial(100, 0.4, size=1000000)

# Оценим вероятность того, что 50 или более заказов стоят дороже 1000
answer = (values >= 50).mean()
print('answer:', round(answer, 3))

answer: 0.027


In [4]:
from scipy import stats

answer = 0
for k in range(50, 101):
    answer += stats.binom.pmf(k, p=0.4, n=100)
print('answer:', round(answer, 3))

answer: 0.027


In [5]:
from scipy import stats

answer = 1 - stats.binom.cdf(49, p=0.4, n=100)
print('answer:', round(answer, 3))

answer: 0.027


#### Ответ: 0.027

###

### Задача 2. Популярные товары
 

Определите топ-3 товара по суммарной выручке. Для решения используйте данные из файла 2022-04-01/2022-04-01T12_df_sales_detail.csv.

#### МОЁ РЕШЕНИЕ:

In [6]:
import pandas as pd

C:\Users\kireevakv\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [11]:
df_sales_detail = pd.read_csv('2022-04-01T12_df_sales_detail.csv')
df_sales_detail['date'] = pd.to_datetime(df_sales_detail['date'])
df_sales_detail.head(3)

,sale_id,good,price,date,user_id
0,1000001,mexican pizza,720,2022-02-04 10:00:24,1c1543
1,1000002,chefs pizza,840,2022-02-04 10:02:28,a9a6e8
2,1000002,orange juice,90,2022-02-04 10:02:28,a9a6e8


In [10]:
df_sales_detail.groupby('good', as_index=False).agg({'price': 'sum'}).sort_values('price', ascending=False).head(3)

,good,price
3,chefs pizza,24558240
6,double pepperoni pizza,22558380
5,chicken bbq pizza,17622540


#### РЕШЕНИЕ ИЗ ЛМС

In [12]:
import os

URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

df_sales_detail = read_database('2022-04-01T12_df_sales_detail.csv')
df_sales_detail['date'] = pd.to_datetime(df_sales_detail['date'])
top_3_revenue = (
    df_sales_detail
    .groupby('good')['price'].sum()
    .sort_values()[-3:].index.tolist()
)
print('Ответ:', top_3_revenue)

Ответ: ['chicken bbq pizza', 'double pepperoni pizza', 'chefs pizza']


### Задача 3. Время от захода на сайт до покупки
  

Оцените средний срок, который проходит от захода пользователя на сайт до совершения покупки. Будем считать, что заход на сайт относится к покупке, если он был совершён не ранее, чем за два часа до совершения покупки. Другими словами, для каждой покупки нужно посчитать время, которое проходит от покупки до первого захода на сайт того же пользователя в течение двух часов перед покупкой. И от полученных значений времени посчитать среднее.

Для решения используйте данные из файлов 2022-04-01T12_df_sales.csv и 2022-04-01T12_df_web_logs.csv.

Полученный результат переведите в минуты и округлите до целого значения.

#### МОЕ РЕШЕНИЕ

In [21]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_web_logs = read_database('2022-04-01T12_df_web_logs.csv')
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])


In [22]:
df_sales.head(2)

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8


In [23]:
df_web_logs.head(2)

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5


In [38]:
base = df_sales.merge(df_web_logs, on=['user_id'], how='left')
orders_dates_info = base[((base['date_x'] - base['date_y']).dt.days == 0) & ((base['date_x'] - base['date_y']).dt.components.hours <= 2)]

In [41]:
sales = orders_dates_info.groupby(['sale_id', 'user_id', 'date_x'], as_index=False).agg({'date_y': 'min'})
sales['diff'] = sales['date_x'] - sales['date_y']

In [47]:
sales['diff'].dt.total_seconds().div(60).mean().round()

17.0

#### РЕШЕНИЕ ЛМС

In [51]:
from datetime import timedelta

# объединяем таблицы с покупками и логами по пользователям
# оставляем только user_id и времена
df = pd.merge(
    df_sales[['user_id', 'date']].rename(columns={'date': 'date_sale'}),
    df_web_logs[['user_id', 'date']].rename(columns={'date': 'date_web'}),
    on='user_id',
    how='inner'
)

# удаляем строки, в которых заход на сайт ранее 2 часов до покупки или после покупки
df = df[
    (df['date_web'] >= df['date_sale'] - timedelta(hours=2))
    & (df['date_web'] < df['date_sale'])
].copy()
# определяем время первого захода на сайт для покупки
df_first_web = df.groupby(['user_id', 'date_sale'])[['date_web']].min().reset_index()
# вычисляем время в секундах
df_first_web['delta'] = (df_first_web['date_sale'] - df_first_web['date_web']).dt.total_seconds()

# считаем среднее время и переводим в минуты
answer = round(df_first_web['delta'].mean() / 60)
print('answer:', answer)

answer: 17


### Задача 4. Знакомство с задачами на код
  

В заданиях на код вам нужно отправить в систему проверки код с решением. Шаблон для решения будет предоставлен в описании задания. Проверка решения запускается в докер-контейнере, в котором доступны следующие версии библиотек:

numpy=1.19.5
pandas==1.2.3
scipy==1.5.4

После окончания проверки отобразятся количество набранных баллов и список пройденных проверок. Если при проверке возникла ошибка, то будет выведено её описание.

Напишите функцию sum_two_values. Описание функции есть в шаблоне решения ниже. Скопируйте код шаблона в окно ввода решения, допишите недостающие части, нажмите кнопку "Отправить решение".

Шаблон решения

__def sum_two_values(a, b):__

    """Возвращает сумму чисел."""
    
    # YOUR_CODE_HERE




In [52]:
def sum_two_values(a, b):
    """Возвращает сумму чисел."""
    return a + b


#### Задача 5. Функция для фильтрации данных
  

Для оценки результатов экспериментов нам нужно выбрать данные, полученные во время эксперимента.

Напишите функцию get_data_subset.

Шаблон решения

In [ ]:
from datetime import datetime

import pandas as pd


def get_data_subset(df, begin_date, end_date, user_ids=None, columns=None):
    """Возвращает подмножество данных.

    :param df (pd.DataFrame): таблица с данными, обязательные столбцы: 'date', 'user_id'.
    :param begin_date (datetime.datetime | None): дата начала интервала с данными.
        Пример, df[df['date'] >= begin_date].
        Если None, то фильтровать не нужно.
    :param end_date (datetime.datetime | None): дата окончания интервала с данными.
        Пример, df[df['date'] < end_date].
        Если None, то фильтровать не нужно.
    :param user_ids (list[str] | None): список user_id, по которым нужно предоставить данные.
        Пример, df[df['user_id'].isin(user_ids)].
        Если None, то фильтровать по user_id не нужно.
    :param columns (list[str] | None): список названий столбцов, по которым нужно предоставить данные.
        Пример, df[columns].
        Если None, то фильтровать по columns не нужно.

    :return df (pd.DataFrame): датафрейм с подмножеством данных.
    """
    # YOUR_CODE_HERE

In [55]:
from datetime import datetime

import pandas as pd


def get_data_subset(df, begin_date, end_date, user_ids=None, columns=None):
    """Возвращает подмножество данных.

    :param df (pd.DataFrame): таблица с данными, обязательные столбцы: 'date', 'user_id'.
    :param begin_date (datetime.datetime | None): дата начала интервала с данными.
        Пример, df[df['date'] >= begin_date].
        Если None, то фильтровать не нужно.
    :param end_date (datetime.datetime | None): дата окончания интервала с данными.
        Пример, df[df['date'] < end_date].
        Если None, то фильтровать не нужно.
    :param user_ids (list[str] | None): список user_id, по которым нужно предоставить данные.
        Пример, df[df['user_id'].isin(user_ids)].
        Если None, то фильтровать по user_id не нужно.
    :param columns (list[str] | None): список названий столбцов, по которым нужно предоставить данные.
        Пример, df[columns].
        Если None, то фильтровать по columns не нужно.

    :return df (pd.DataFrame): датафрейм с подмножеством данных.
    """
    df['date'] = pd.to_datetime(df['date'])
    
    if begin_date is not None:
        df = df[df['date']> begin_date]
    
    if end_date is not None:
        df = df[df['date'] < end_date]
    
    if user_ids is not None:
        df = df[df['user_id'].isin(user_ids)]
        
    if columns is not None:
        return df[columns]

#### Задача 6. Функции вычисления метрик
 

Напишите функции get_response_time, get_revenue_web и get_revenue_all для вычисления метрик «revenue (web)», «revenue (all)» и «response time».

In [ ]:
# Шаблон решения

import pandas as pd

from datetime import datetime


def get_response_time(df_web_logs, begin_date, end_date):
    """Вычисляет значения времени обработки запроса сервером.

    Нужно вернуть значения user_id и load_time из таблицы df_web_logs,
    отфильтрованные по дате.
    Считаем, что запросы обрабатываются независимо, поэтому группировать
    по user_id не нужно.

    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
    столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для
    фильтрации данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    # YOUR_CODE_HERE


def get_revenue_web(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт в указанный период.

    Эти данные нужны для экспериментов на сайте, когда в эксперимент
    попадают только те, кто заходил на сайт во время эксперимента.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный
    период для пользователей, заходивших на сайт в указанный период.
    Если пользователь зашёл на сайт и ничего не купил, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    # YOUR_CODE_HERE


def get_revenue_all(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт до end_date.

    Эти данные нужны, например, для экспериментов с рассылкой по email,
    когда в эксперимент попадают те, кто когда-либо оставил нам свои данные.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный период
    для пользователей, заходивших на сайт до end_date.
    Если пользователь ничего не купил за указанный период, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    # YOUR_CODE_HERE

#### МОЕ РЕШЕНИЕ

In [77]:
import pandas as pd

from datetime import datetime


def get_response_time(df_web_logs, begin_date, end_date):
    """Вычисляет значения времени обработки запроса сервером.

    Нужно вернуть значения user_id и load_time из таблицы df_web_logs,
    отфильтрованные по дате.
    Считаем, что запросы обрабатываются независимо, поэтому группировать
    по user_id не нужно.

    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
    столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для
    фильтрации данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])
    
    if begin_date:
        df_web_logs = df_web_logs[(df_web_logs['date'] >= begin_date)]
    if end_date:
        df_web_logs = df_web_logs[(df_web_logs['date'] < end_date)]
        
    res = df_web_logs[['user_id', 'load_time']].rename(columns={'load_time': 'metric'})
        
    return res


def get_revenue_web(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт в указанный период.

    Эти данные нужны для экспериментов на сайте, когда в эксперимент
    попадают только те, кто заходил на сайт во время эксперимента.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный
    период для пользователей, заходивших на сайт в указанный период.
    Если пользователь зашёл на сайт и ничего не купил, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    df_sales['date'] = pd.to_datetime(df_sales['date'])
    
    visitors = get_response_time(df_web_logs, begin_date, end_date)[['user_id']].drop_duplicates()
        
    if begin_date:
        df_sales = df_sales[(df_sales['date'] >= begin_date)]
    if end_date:
        df_sales = df_sales[(df_sales['date'] < end_date)]
    
    buyers = df_sales.groupby('user_id', as_index=False).agg({'price': 'sum'}).rename(columns={'price': 'metric'})
    
    visitors_purchases = visitors.merge(buyers, on='user_id', how='outer').fillna(0)
    
    return visitors_purchases

def get_revenue_all(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт до end_date.

    Эти данные нужны, например, для экспериментов с рассылкой по email,
    когда в эксперимент попадают те, кто когда-либо оставил нам свои данные.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный период
    для пользователей, заходивших на сайт до end_date.
    Если пользователь ничего не купил за указанный период, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    df_sales['date'] = pd.to_datetime(df_sales['date'])
    
    visitors = get_response_time(df_web_logs, None, end_date)[['user_id']].drop_duplicates()
        
    if begin_date:
        df_sales = df_sales[(df_sales['date'] >= begin_date)]
    if end_date:
        df_sales = df_sales[(df_sales['date'] < end_date)]
    
    buyers = df_sales.groupby('user_id', as_index=False).agg({'price': 'sum'}).rename(columns={'price': 'metric'})
    
    visitors_purchases = visitors.merge(buyers, on='user_id', how='outer').fillna(0)
    
    return visitors_purchases

#### РЕШЕНИЕ ЛМС

In [ ]:
import pandas as pd


def get_data_subset(df, begin_date, end_date, user_ids=None, columns=None):
    if begin_date:
        df = df[df['date'] >= begin_date]
    if end_date:
        df = df[df['date'] < end_date]
    if user_ids:
        df = df[df['user_id'].isin(user_ids)]
    if columns:
        df = df[columns]
    return df.copy()


def get_response_time(df_web_logs, begin_date, end_date):
    return (
        get_data_subset(df_web_logs, begin_date, end_date, None, ['user_id', 'load_time'])
        .rename(columns={'load_time': 'metric'})
        [['user_id', 'metric']]
    )


def get_revenue_web(df_sales, df_web_logs, begin_date, end_date):
    df_users = (
        get_data_subset(df_web_logs, begin_date, end_date, None, ['user_id'])
        .drop_duplicates()
    )
    df = (
        get_data_subset(df_sales, begin_date, end_date, None, ['user_id', 'price'])
        .groupby('user_id')[['price']].sum().reset_index() 
        .rename(columns={'price': 'metric'})
    )
    df = pd.merge(df_users, df, on='user_id', how='left').fillna(0)
    return df[['user_id', 'metric']]


def get_revenue_all(df_sales, df_web_logs, begin_date, end_date):
    df_users = (
        get_data_subset(df_web_logs, None, end_date, None, ['user_id'])
        .drop_duplicates()
    )
    df = (
        get_data_subset(df_sales, begin_date, end_date, None, ['user_id', 'price'])
        .groupby('user_id')[['price']].sum().reset_index() 
        .rename(columns={'price': 'metric'})
    )
    df = pd.merge(df_users, df, on='user_id', how='left').fillna(0)
    return df[['user_id', 'metric']]
